In [5]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import shapely
from shapely.geometry import mapping, Polygon, MultiPoint, MultiPolygon, shape
import fiona
import os
import shutil
import random
from shapely.geometry import MultiPolygon

In [166]:
arbutusObsRaw = pd.read_csv("ArbutusObsevations.csv", sep = "\t")
arbutusObs = arbutusObsRaw[['species', 'decimalLatitude', 'decimalLongitude', 'coordinateUncertaintyInMeters', 'coordinatePrecision', 'elevation']]
arbutusObs = arbutusObs.dropna(subset=['species'])

/tmp/ipykernel_3511/2179390862.py:1: DtypeWarning: Columns (10,39,46) have mixed types. Specify dtype option on import or set low_memory=False.
  arbutusObsRaw = pd.read_csv("ArbutusObsevations.csv", sep = "\t")


In [81]:
def extractCoords(obsFrame):
    coords = {}
    speciesNames = obsFrame["species"].unique()
    for species in speciesNames:
        currSpeciesObs = obsFrame[obsFrame["species"] == species]
        if currSpeciesObs.index.size == 1:
                currSpeciesPts = [[currSpeciesObs["decimalLongitude"].iloc(0)[i],currSpeciesObs["decimalLatitude"].iloc(0)[i]] for i in range(len(currSpeciesObs["decimalLongitude"]))]
        else:
            latMean = currSpeciesObs["decimalLatitude"].astype(int).mean()
            lonMean = currSpeciesObs["decimalLongitude"].astype(int).mean()
            latStd = currSpeciesObs["decimalLatitude"].astype(int).std()
            lonStd = currSpeciesObs["decimalLongitude"].astype(int).std()
            currSpeciesObs = currSpeciesObs[np.abs(currSpeciesObs["decimalLatitude"].astype(int) - latMean)/(latStd + .1) < 5]
            currSpeciesObs = currSpeciesObs[np.abs(currSpeciesObs["decimalLongitude"].astype(int) - lonMean)/(lonStd + .1) < 5]
            currSpeciesPts = [[currSpeciesObs["decimalLongitude"].iloc(0)[i],currSpeciesObs["decimalLatitude"].iloc(0)[i]] for i in range(len(currSpeciesObs["decimalLongitude"]))]
        coords[species] = currSpeciesPts
    return coords


In [8]:
def minDist(l1,l2):
    M = 9999
    for pt1 in l1:
       for pt2 in l2:
           M = min(math.dist(pt1,pt2), M)
    return M

In [9]:
def minDistRandom(l1,l2):
    M = 9999
    numSample = 100
#Generate 5 random numbers between 10 and 30
    randomlist1 = random.sample(l1, min(numSample, len(l1)))
    randomlist2 = random.sample(l2, min(numSample, len(l2)))
    
    for pt1 in randomlist1:
       for pt2 in randomlist2:
           M = min(math.dist(pt1,pt2), M)
    return M

In [100]:
def myClustering(points, maxDist):
    populations = []
    sortedPoints = sorted(points , key=lambda k: [k[1], k[0]])
    minPop = []
    minPop.append(sortedPoints[0])
    populations.append(minPop)
    counter = 0
    maxMinDist = 0
    for pt in sortedPoints: 
        counter += 1
        print(counter)
        D = minDist([pt], populations[0])
        for pop in populations:
            D2 = minDist([pt], pop)
            maxMinDist = max(maxMinDist, D2)
            if D2 < D:
                minPop = pop
                D = D2
        if D2 < maxDist:
            minPop.append(pt)
        else: 
            populations.append([pt])
    print(maxMinDist)
        #compute the mininum distance to a population
 #If this minimum distance is bigger than max distance, make a new population. 
    return populations

In [113]:
def myClusteringFast(points, maxDist):
    populations = []
    populationsDict = {}
    sortedPoints = sorted(points , key=lambda k: [k[0], k[1]])

    maxMinDist = 0
    counter = 0
    while counter < len(sortedPoints):
        currPoint = sortedPoints[counter]
        D = maxDist
        minIndex = -1
        for i in range(1,min(counter,10)):
            D2 = minDistRandom([currPoint], [sortedPoints[counter - i]])
            if  D2 < D:
                D = D2
                maxMinDist = max(D2,maxMinDist) 
                minIndex = counter - i
        if minIndex != -1:
            popIndex = populationsDict[minIndex]
            populations[popIndex].append(currPoint)
            populationsDict[counter] = popIndex
        else: 
            populations.append([currPoint])
            populationsDict[counter] = len(populations) - 1
        counter = counter + 1
        #compute the mininum distance to a population
 #If this minimum distance is bigger than max distance, make a new population. 
    return populations

In [12]:
def postProcessing(populations, maxDist):
    newPopulations = []
    for i in range(len(populations)):
        j = 0
        while j < len(newPopulations):
            if maxDist > minDist(populations[i], newPopulations[j]):
                newPopulations[j] = newPopulations[j] + populations[i]
                break
            j = j + 1
        if j == len(newPopulations):
            newPopulations.append(populations[i])
    return newPopulations


    # for each pop in populations
    #loop through the previous populations
    #If one is too close, combine the two and add it to the, 'toy' question
    #return the new population

In [13]:
def postProcessingRandom(populations, maxDist):
    newPopulations = []
    counter = 0 
    for i in range(len(populations)):
        j = 0
        while j < len(newPopulations):
            if maxDist > minDistRandom(populations[i], newPopulations[j]):
                newPopulations[j] = newPopulations[j] + populations[i]
                break
            j = j + 1
        if j == len(newPopulations):
            newPopulations.append(populations[i])
        counter = counter + 1
    return newPopulations


    # for each pop in populations
    #loop through the previous populations
    #If one is too close, combine the two and add it to the list
    #If none is close enough just add the population in question
    #return the new population

In [94]:
def formPopulations(coordDictionary, maxDist):
    populationDict = {}
    for species in coordDictionary.keys():
        currPopulationsPre = myClusteringFast(coordDictionary[species], maxDist)
        currPopulationsPost = postProcessingRandom(currPopulationsPre,maxDist)
        prevLength = 0
        while len(currPopulationsPost) != prevLength:
            prevLength = len(currPopulationsPost) 
            currPopulationsPost = postProcessingRandom(currPopulationsPost, maxDist)
        populationDict[species] = postProcessing(currPopulationsPost, maxDist)
    return populationDict

In [89]:
def showPopulations(populationDict, width):
    fig, axs = plt.subplots(len(populationDict.keys()), figsize=(width, width*len(populationDict.keys())))
    fig.suptitle('Vertically stacked subplots')

    counter = 0
    for species in populationDict.keys():
        populations = populationDict[species]
        for i in range(len(populations)):
            axs[counter].plot(np.array(populations[i])[:,0], np.array(populations[i])[:,1], 'o')
        counter = counter + 1

In [158]:
def formPolygons(populationDict, buffRad):
    bufferedPolysDict = {}
    for species in populationDict.keys():
        specPolys = []
        for pop in populationDict[species]:
            specPolys.append(shapely.concave_hull(MultiPoint(pop), ratio=0.1))
        bufferedPolysList = []
        for poly in specPolys:
            bufferedPolysList.append(shapely.buffer(poly, buffRad, 2))
        bufferedPolysDict[species] = MultiPolygon(bufferedPolysList)
    return bufferedPolysDict


In [162]:
def savePolygons(polyDict):
    schema = {
        'geometry': 'MultiPolygon',
        'properties': {'id': 'int'},
    }

    counter = 0
    print(polyDict.keys())
    for species in polyDict.keys():
        dirName = 'shapes/' + species
        fileName = dirName + '/' + species + 'shp' + '.shp'
        os.makedirs(dirName)
        with fiona.open(fileName, 'w', 'ESRI Shapefile', schema) as c:
        ## If there are multiple geometries, put the "for" loop here
            c.write({
                'geometry': mapping(polyDict[species]),
                'properties': {'id': 122},
            })
        shutil.make_archive('shapes/' + species, 'zip', dirName)




In [167]:
coordDict = extractCoords(arbutusObs)

In [168]:
populationDict = formPopulations(coordDict, .9)

In [169]:
polyDict = formPolygons(populationDict, .25)

In [171]:
savePolygons(polyDict)

dict_keys(['Arbutus unedo', 'Arbutus menziesii', 'Arbutus arizonica', 'Arbutus madrensis', 'Arbutus xalapensis', 'Arbutus bicolor', 'Arbutus andrachne', 'Arbutus tessellata', 'Arbutus canariensis', 'Arbutus mollis', 'Arbutus andrachnoides', 'Arbutus androsterilis'])


In [19]:
unedoPolysBoundaries = []
for poly in unedoPolys:
    if poly.geom_type == "Polygon":
        unedoPolysBoundaries.append(list(poly.exterior.coords))
    if poly.geom_type == "Point":
        unedoPolysBoundaries.append(list(poly.coords))
    if poly.geom_type == "LineString":
        unedoPolysBoundaries.append(list(poly.coords))

NameError: name 'unedoPolys' is not defined

In [ ]:
bufferedBoundaries = []
rad = .5
for poly in unedoPolysBoundaries:
    currBuffBound = []
    for pt in poly:
        currBuffBound.append(list(pt))
        currBuffBound.append([pt[0] + rad, pt[1]])
        currBuffBound.append([pt[0] , pt[1] + rad])
        currBuffBound.append([pt[0] - rad, pt[1]])
        currBuffBound.append([pt[0], pt[1] - rad])
    bufferedBoundaries.append(currBuffBound)

In [ ]:
bufferedPolys = []
for pop in bufferedBoundaries:
    bufferedPolys.append(shapely.concave_hull(MultiPoint(pop), ratio=0.1))

NameError: name 'bufferedBoundaries' is not defined

In [ ]:
Multi = MultiPolygon(bufferedAttempt2) 

'/home/fenton/Documents/Coding/madroneRange/madroneRange/shapes/combined_unedo_shpZipped.zip'

In [ ]:
schema = {
    'geometry': 'Polygon',
    'properties': {'id': 'int'},
}

counter = 0
for polygon in bufferedAttempt2:
    dirHead = 'shapes/my_unedo_shp'
    dirName = 'shapes/my_unedo_shp' + str(counter)
    fileName = dirName + '/my_unedo_shp' + str(counter) + '.shp'
    os.makedirs(dirName)
    with fiona.open(fileName, 'w', 'ESRI Shapefile', schema) as c:
    ## If there are multiple geometries, put the "for" loop here
        c.write({
            'geometry': mapping(polygon),
            'properties': {'id': 123},
        })
    shutil.make_archive('shapes/unedoBuffThree' + str(counter) + 'Zipped', 'zip', dirName)
    counter = counter + 1